In [3]:
import sys
sys.path.insert(1, '/Users/madisonthantu/Desktop/DREAM/t-recs')
from trecs.metrics import Measurement, MSEMeasurement, InteractionSpread, InteractionSpread, InteractionSimilarity, RecSimilarity, RMSEMeasurement, InteractionMeasurement
from trecs.components import Users

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from collections import defaultdict

sys.path.insert(1, '/Users/madisonthantu/Desktop/DREAM/T-RECS-RS-research')
from wrapper.models.bubble import BubbleBurster
from wrapper.metrics.evaluation_metrics import TopicInteractionMeasurement
from src.utils import compute_constrained_clusters, create_global_user_pairs, user_topic_mapping, create_cluster_user_pairs, load_and_process_movielens, compute_embeddings
from wrapper.metrics.clustering_metrics import MeanCosineSim, MeanDistanceFromCentroid, MeanCosineSimPerCluster, MeanDistanceFromCentroidPerCluster
from prelim_experiments.param_experiments.chaney_utils import *

import warnings
warnings.simplefilter("ignore")

import itertools

In [4]:
num_attributes=15
max_iter=1000
num_clusters = 10

data_path = '/Users/madisonthantu/Desktop/DREAM/data/ml-100k/u.data'
binary_ratings_matrix = load_and_process_movielens(file_path=data_path)
user_representation, item_representation = compute_embeddings(binary_ratings_matrix, n_attrs=num_attributes, max_iter=max_iter)

item_cluster_ids, item_cluster_centers = compute_constrained_clusters(embeddings=item_representation.T, name='item_clusters', n_clusters=num_clusters)
user_cluster_ids, user_cluster_centers = compute_constrained_clusters(embeddings=user_representation, name='user_clusters', n_clusters=num_clusters)
inter_cluster_user_pairs, intra_cluster_user_pairs = create_cluster_user_pairs(user_cluster_ids)
                
users = Users(
    actual_user_profiles=user_representation, 
    repeat_interactions=False,
    drift=0.05,
    attention_exp=-0.2
)

Calculating embeddings ...
... Calculated embeddings.
Calculating constrained clusters ...
... Calculated constrained clusters.
Calculating constrained clusters ...
... Calculated constrained clusters.


In [5]:
metrics_list = [
    MSEMeasurement(diagnostics=True),
    MeanCosineSim(pairs=intra_cluster_user_pairs, name='mean_intra_cluster_cosine_sim', diagnostics=True),
    InteractionSimilarity(pairs=intra_cluster_user_pairs, name="test_interaction_similarity", diagnostics=True),
    InteractionSpread(),
    InteractionMeasurement(),
    TopicInteractionMeasurement()
]

bubble = BubbleBurster(
    actual_user_representation=users, 
    actual_item_representation=item_representation,
    item_topics=item_cluster_ids,
    num_attributes=num_attributes,
    num_items_per_iter=10,
    record_base_state=True
)

In [6]:
train_timesteps = 5
run_timesteps = 10
repeated_training = 1

bubble.add_metrics(*metrics_list)
bubble.startup_and_train(timesteps=train_timesteps)
bubble.run(timesteps=run_timesteps, train_between_steps=repeated_training)
bubble.close() # end logging

100%|██████████| 10/10 [00:15<00:00,  1.55s/it]


In [ ]:
class TopicInteractionMeasurement(Measurement):
    """
    Keeps track of the interactions between users and items.

    Specifically, at each timestep, it stores a histogram of length
    :math:`|I|`, where element :math:`i` is the number of interactions
    received by item :math:`i`.

    Parameters
    -----------

        verbose: bool, default False
            If ``True``, enables verbose mode. Disabled by default.

    Attributes
    -----------
        Inherited by Measurement: :class:`.Measurement`

        name: str, default ``"interaction_histogram"``
            Name of the measurement component.
    """

    def __init__(self, name="topic_interaction_histogram", verbose=False):
        Measurement.__init__(self, name, verbose)

    @staticmethod
    def _generate_interaction_histogram(interactions, num_users, num_items):
        """
        Generates a histogram of the number of interactions per item at the
        given timestep.

        Parameters
        -----------
            interactions : :obj:`numpy.ndarray`
                Array of user interactions.

            num_users : int
                Number of users in the system

            num_items : int
                Number of items in the system

        Returns
        ---------
            :obj:`numpy.ndarray`:
                Histogram of the number of interactions aggregated by items at the given timestep.
        """
        histogram = np.zeros(num_items)
        np.add.at(histogram, interactions, 1)
        # Check that there's one interaction per user
        if histogram.sum() != num_users:
            raise ValueError("The sum of interactions must be equal to the number of users")
        return histogram

    def measure(self, recommender):
        """
        Measures and stores a histogram of the number of interactions per
        item at the given timestep.

        Parameters
        ------------
            recommender: :class:`~models.recommender.BaseRecommender`
                Model that inherits from :class:`~models.recommender.BaseRecommender`.
        """
        if recommender.interactions.size == 0:
            # at beginning of simulation, there are no interactions
            self.observe(None)
            return

        histogram = self._generate_interaction_histogram(
            recommender.interactions, recommender.num_users, recommender.num_items
        )
        self.observe(histogram, copy=True)